# Classificação com NaiveBayes:

- modela atributo sobre a classe de forma independente, ie como se um atributo n influenciasse outro
- é extremamente performático (leve), e bastante eficiente apesar da condição acima

# Hiper Parâmetros:

- modelType: multinomial (default), bernoulli ouy gaussian. 

In [2]:
import findspark, pyspark
from pyspark.sql import SparkSession
findspark.init()
spark = SparkSession.builder.appName("naive").getOrCreate()

In [5]:
iris = spark.read.csv("iris.csv", header=True, inferSchema=True, sep=",")
iris.show(5)
print(iris.count())

+-----------+----------+-----------+----------+-----------+
|sepallength|sepalwidth|petallength|petalwidth|      class|
+-----------+----------+-----------+----------+-----------+
|        5.1|       3.5|        1.4|       0.2|Iris-setosa|
|        4.9|       3.0|        1.4|       0.2|Iris-setosa|
|        4.7|       3.2|        1.3|       0.2|Iris-setosa|
|        4.6|       3.1|        1.5|       0.2|Iris-setosa|
|        5.0|       3.6|        1.4|       0.2|Iris-setosa|
+-----------+----------+-----------+----------+-----------+
only showing top 5 rows

150


In [6]:
from pyspark.ml.feature import RFormula
Rformula = RFormula(formula="class ~ .", featuresCol="independente", labelCol="dependente")
irisrf = Rformula.fit(iris).transform(iris)
irisrf.select("independente","dependente").show(5)
# Categorical encoding realizado na label (resposta = y): iris-setosa=0

+-----------------+----------+
|     independente|dependente|
+-----------------+----------+
|[5.1,3.5,1.4,0.2]|       0.0|
|[4.9,3.0,1.4,0.2]|       0.0|
|[4.7,3.2,1.3,0.2]|       0.0|
|[4.6,3.1,1.5,0.2]|       0.0|
|[5.0,3.6,1.4,0.2]|       0.0|
+-----------------+----------+
only showing top 5 rows



In [7]:
irisTreino, irisTeste = irisrf.randomSplit([0.7,0.3])
print(irisTreino.count())
print(irisTeste.count())

106
44


In [8]:
from pyspark.ml.classification import NaiveBayes
naive = NaiveBayes(smoothing=1.0, modelType="multinomial", featuresCol="independente", labelCol="dependente")
modelo = naive.fit(irisTreino)

In [9]:
previsao = modelo.transform(irisTeste)
previsao.select("dependente","prediction").show(20)

+----------+----------+
|dependente|prediction|
+----------+----------+
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       1.0|       1.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       1.0|       1.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       1.0|       1.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       1.0|       2.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       1.0|       2.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
+----------+----------+
only showing top 20 rows



In [14]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
performance = MulticlassClassificationEvaluator(labelCol="dependente", predictionCol="prediction", 
                                                metricName="accuracy")
acuracia = performance.evaluate(previsao)
print(acuracia)

0.8863636363636364
